In [110]:
from openai.types.responses.response_input_param import FunctionCallOutput
from openai.types.responses import ResponseFunctionToolCall, FunctionTool, WebSearchTool, FunctionToolParam, WebSearchToolParam
from openai import OpenAI

In [97]:
client = OpenAI()

In [98]:
def handle_tool_call(tool_call: ResponseFunctionToolCall) -> FunctionCallOutput:
    tool_call_id = tool_call.call_id
    function_name = tool_call.name
    arguments_dict = eval(tool_call.arguments)

    print(f"Tool call ID: {tool_call_id}")
    print(f"Function name: {function_name}")
    print(f"Arguments dict: {arguments_dict}")

    result = getattr(globals()[function_name], "__call__")(**arguments_dict)

    # Check if result is a string or an object with model_dumps_json method
    if isinstance(result, str):
        # If it's a string, use it directly
        output = result
    elif hasattr(result, "model_dump"):
        # If it's an object with model_dump method, use that
        output = result.model_dump()
    else:
        # For any other type, convert to string
        output = str(result)

    return FunctionCallOutput(
        call_id=tool_call_id,
        output=output,
        type="function_call_output"
    )

In [108]:
def green_colour_attached_to_name(name: str) -> str:
    return f"{name}, you are now Green :)"


green_colour_attached_to_name_tool = FunctionTool(
    type="function",
    name="green_colour_attached_to_name",
    description="Use this to attach word 'Green' to the provided name.",
    parameters={
        "type": "object",
        "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the person",
                },
        },
        "required": ["name"],
        "additionalProperties": False,
    },
    strict=True,
)

web_search_tool = WebSearchTool(type="web_search_preview")

In [109]:
from typing import cast
green_colour_attached_to_name_tool_param = cast(
    FunctionToolParam, green_colour_attached_to_name_tool)
web_search_tool_param = cast(WebSearchToolParam, web_search_tool)

In [106]:
response = client.responses.create(
    model="gpt-4o",
    instructions="You are a helpful assistant.",
    input="Attach a colour to the my name.",
    tools=[green_colour_attached_to_name_tool_param, web_search_tool_param]
)

TypeError: Object of type function is not JSON serializable

In [85]:
response.model_dump()

{'id': 'resp_67d31cdbf2088191b6560c51ee58598d036dbb28070fa328',
 'created_at': 1741888731.0,
 'error': None,
 'incomplete_details': None,
 'instructions': 'You are a helpful assistant.',
 'metadata': {},
 'model': 'gpt-4o-2024-08-06',
 'object': 'response',
 'output': [{'id': 'msg_67d31cdc52d48191bddfe2c4e0510312036dbb28070fa328',
   'content': [{'annotations': [],
     'text': "Sure, tell me your name and the color you'd like!",
     'type': 'output_text'}],
   'role': 'assistant',
   'status': 'completed',
   'type': 'message'}],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [{'name': 'green_colour_attached_to_name',
   'parameters': {'type': 'object',
    'properties': {'name': {'type': 'string',
      'description': 'The name of the person'}},
    'required': ['name'],
    'additionalProperties': False},
   'strict': True,
   'type': 'function',
   'description': "Use this to attach word 'Green' to the provided name."},
  {'type': 'web_search_

In [86]:
new_response = client.responses.create(
    model="gpt-4o",
    instructions="You are a helpful assistant.",
    input="My name is Jonathan Joster.",
    previous_response_id=response.id,
    tools=[green_colour_attached_to_name_tool_param, web_search_tool_param]
)

In [87]:
new_response.model_dump()

{'id': 'resp_67d31cdccfd88191905ccf63b9b322ce036dbb28070fa328',
 'created_at': 1741888733.0,
 'error': None,
 'incomplete_details': None,
 'instructions': 'You are a helpful assistant.',
 'metadata': {},
 'model': 'gpt-4o-2024-08-06',
 'object': 'response',
 'output': [{'id': 'fc_67d31cddb8448191ae88b1d2273d2f9e036dbb28070fa328',
   'arguments': '{"name":"Jonathan Joster"}',
   'call_id': 'call_oFXiPh4fNj1DKKnG3kek2bcW',
   'name': 'green_colour_attached_to_name',
   'type': 'function_call',
   'status': 'completed'}],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [{'name': 'green_colour_attached_to_name',
   'parameters': {'type': 'object',
    'properties': {'name': {'type': 'string',
      'description': 'The name of the person'}},
    'required': ['name'],
    'additionalProperties': False},
   'strict': True,
   'type': 'function',
   'description': "Use this to attach word 'Green' to the provided name."},
  {'type': 'web_search_preview',
   

In [92]:
for tool_call in new_response.output:
    if tool_call.type != "function_call":
        continue
    tool_result_message = handle_tool_call(tool_call)

17:59:36.592 Tool call ID: call_oFXiPh4fNj1DKKnG3kek2bcW
17:59:36.592 Function name: green_colour_attached_to_name
17:59:36.592 Arguments dict: {'name': 'Jonathan Joster'}


In [89]:
new_response_tool_result = client.responses.create(
    model="gpt-4o",
    instructions="You are a helpful assistant.",
    input=[tool_result_message],
    previous_response_id=new_response.id,
    tools=[green_colour_attached_to_name_tool_param, web_search_tool_param]
)

In [90]:
new_response_tool_result.model_dump()

{'id': 'resp_67d31cde17348191b1f70b8d8c79b75b036dbb28070fa328',
 'created_at': 1741888734.0,
 'error': None,
 'incomplete_details': None,
 'instructions': 'You are a helpful assistant.',
 'metadata': {},
 'model': 'gpt-4o-2024-08-06',
 'object': 'response',
 'output': [{'id': 'msg_67d31cdeb7e48191bb6e226d2cdd3ca2036dbb28070fa328',
   'content': [{'annotations': [],
     'text': 'Jonathan Joster, you are now associated with the color Green! 🌿',
     'type': 'output_text'}],
   'role': 'assistant',
   'status': 'completed',
   'type': 'message'}],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [{'name': 'green_colour_attached_to_name',
   'parameters': {'type': 'object',
    'properties': {'name': {'type': 'string',
      'description': 'The name of the person'}},
    'required': ['name'],
    'additionalProperties': False},
   'strict': True,
   'type': 'function',
   'description': "Use this to attach word 'Green' to the provided name."},
  {'type'

In [91]:
client.responses.input_items.list(
    new_response_tool_result.id).model_dump()["data"][::-1]

[{'id': 'msg_67d31cdbf32c81918889bcdf2dda249b036dbb28070fa328',
  'content': [{'text': 'Attach a colour to the my name.',
    'type': 'input_text'}],
  'role': 'user',
  'status': 'completed',
  'type': 'message'},
 {'id': 'msg_67d31cdc52d48191bddfe2c4e0510312036dbb28070fa328',
  'content': [{'annotations': [],
    'text': "Sure, tell me your name and the color you'd like!",
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'},
 {'id': 'msg_67d31cdcd118819191c8376f0ada5692036dbb28070fa328',
  'content': [{'text': 'My name is Jonathan Joster.', 'type': 'input_text'}],
  'role': 'user',
  'status': 'completed',
  'type': 'message'},
 {'id': 'fc_67d31cddb8448191ae88b1d2273d2f9e036dbb28070fa328',
  'arguments': '{"name":"Jonathan Joster"}',
  'call_id': 'call_oFXiPh4fNj1DKKnG3kek2bcW',
  'name': 'green_colour_attached_to_name',
  'type': 'function_call',
  'status': 'completed'},
 {'id': 'fco_67d31cde18408191abab9ee7ad3fffef036dbb28070fa328',
  